In [2]:
import torch
import math

In [53]:
max_tokens = 12
tokens_per_sample = 5
num_heads = 4

In [54]:
start = (2**(-2**-(math.log2(num_heads)-3)))
start

0.25

In [55]:
ratio = start
slopes = [start*ratio**i for i in range(num_heads)]
slopes

[0.25, 0.0625, 0.015625, 0.00390625]

In [56]:
slopes_t = torch.Tensor(slopes) * -1
slopes_t

tensor([-0.2500, -0.0625, -0.0156, -0.0039])

In [57]:
termo_1 = slopes_t.unsqueeze(1).unsqueeze(1)
print(termo_1.shape)
termo_1

torch.Size([4, 1, 1])


tensor([[[-0.2500]],

        [[-0.0625]],

        [[-0.0156]],

        [[-0.0039]]])

In [58]:
termo_2 = torch.arange(tokens_per_sample).unsqueeze(0).unsqueeze(0).expand(num_heads, -1, -1)
print(termo_2.shape)
termo_2

torch.Size([4, 1, 5])


tensor([[[0, 1, 2, 3, 4]],

        [[0, 1, 2, 3, 4]],

        [[0, 1, 2, 3, 4]],

        [[0, 1, 2, 3, 4]]])

In [63]:
alibi = termo_1 * termo_2
print(alibi.shape)
alibi

torch.Size([4, 1, 5])


tensor([[[-0.0000, -0.2500, -0.5000, -0.7500, -1.0000]],

        [[-0.0000, -0.0625, -0.1250, -0.1875, -0.2500]],

        [[-0.0000, -0.0156, -0.0312, -0.0469, -0.0625]],

        [[-0.0000, -0.0039, -0.0078, -0.0117, -0.0156]]])

In [64]:
alibi, _ = torch.sort(alibi, dim=-1, descending=False)
alibi

tensor([[[-1.0000, -0.7500, -0.5000, -0.2500, -0.0000]],

        [[-0.2500, -0.1875, -0.1250, -0.0625, -0.0000]],

        [[-0.0625, -0.0469, -0.0312, -0.0156, -0.0000]],

        [[-0.0156, -0.0117, -0.0078, -0.0039, -0.0000]]])

In [65]:
alibi = alibi.view(num_heads, 1, tokens_per_sample)
alibi

tensor([[[-1.0000, -0.7500, -0.5000, -0.2500, -0.0000]],

        [[-0.2500, -0.1875, -0.1250, -0.0625, -0.0000]],

        [[-0.0625, -0.0469, -0.0312, -0.0156, -0.0000]],

        [[-0.0156, -0.0117, -0.0078, -0.0039, -0.0000]]])

In [66]:
# alibi = alibi.repeat(max_tokens // tokens_per_sample, 1, 1)
# alibi

In [67]:
def fill_with_neg_inf(t):
    """FP16-compatible function that fills a tensor with -inf."""
    return t.float().fill_(float("-inf")).type_as(t)

In [68]:
future_mask = torch.triu(fill_with_neg_inf(torch.zeros((tokens_per_sample, tokens_per_sample))), diagonal=1)
future_mask

tensor([[0., -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0.]])

In [70]:
future_mask = future_mask.unsqueeze(0) + alibi
future_mask

tensor([[[[-2.0000,    -inf,    -inf,    -inf,    -inf],
          [-2.0000, -1.5000,    -inf,    -inf,    -inf],
          [-2.0000, -1.5000, -1.0000,    -inf,    -inf],
          [-2.0000, -1.5000, -1.0000, -0.5000,    -inf],
          [-2.0000, -1.5000, -1.0000, -0.5000,  0.0000]],

         [[-0.5000,    -inf,    -inf,    -inf,    -inf],
          [-0.5000, -0.3750,    -inf,    -inf,    -inf],
          [-0.5000, -0.3750, -0.2500,    -inf,    -inf],
          [-0.5000, -0.3750, -0.2500, -0.1250,    -inf],
          [-0.5000, -0.3750, -0.2500, -0.1250,  0.0000]],

         [[-0.1250,    -inf,    -inf,    -inf,    -inf],
          [-0.1250, -0.0938,    -inf,    -inf,    -inf],
          [-0.1250, -0.0938, -0.0625,    -inf,    -inf],
          [-0.1250, -0.0938, -0.0625, -0.0312,    -inf],
          [-0.1250, -0.0938, -0.0625, -0.0312,  0.0000]],

         [[-0.0312,    -inf,    -inf,    -inf,    -inf],
          [-0.0312, -0.0234,    -inf,    -inf,    -inf],
          [-0.0312, -0.02

In [71]:
future_mask.shape

torch.Size([1, 4, 5, 5])

In [74]:
future_mask[:, :, :3, :3]

tensor([[[[-2.0000,    -inf,    -inf],
          [-2.0000, -1.5000,    -inf],
          [-2.0000, -1.5000, -1.0000]],

         [[-0.5000,    -inf,    -inf],
          [-0.5000, -0.3750,    -inf],
          [-0.5000, -0.3750, -0.2500]],

         [[-0.1250,    -inf,    -inf],
          [-0.1250, -0.0938,    -inf],
          [-0.1250, -0.0938, -0.0625]],

         [[-0.0312,    -inf,    -inf],
          [-0.0312, -0.0234,    -inf],
          [-0.0312, -0.0234, -0.0156]]]])

In [75]:
mascara_normal = (
            torch.tril(torch.ones((5, 5)))
            .view((1, 1, 5, 5))
        )
mascara_normal

tensor([[[[1., 0., 0., 0., 0.],
          [1., 1., 0., 0., 0.],
          [1., 1., 1., 0., 0.],
          [1., 1., 1., 1., 0.],
          [1., 1., 1., 1., 1.]]]])

In [76]:
mascara_normal + future_mask

tensor([[[[-1.0000,    -inf,    -inf,    -inf,    -inf],
          [-1.0000, -0.5000,    -inf,    -inf,    -inf],
          [-1.0000, -0.5000,  0.0000,    -inf,    -inf],
          [-1.0000, -0.5000,  0.0000,  0.5000,    -inf],
          [-1.0000, -0.5000,  0.0000,  0.5000,  1.0000]],

         [[ 0.5000,    -inf,    -inf,    -inf,    -inf],
          [ 0.5000,  0.6250,    -inf,    -inf,    -inf],
          [ 0.5000,  0.6250,  0.7500,    -inf,    -inf],
          [ 0.5000,  0.6250,  0.7500,  0.8750,    -inf],
          [ 0.5000,  0.6250,  0.7500,  0.8750,  1.0000]],

         [[ 0.8750,    -inf,    -inf,    -inf,    -inf],
          [ 0.8750,  0.9062,    -inf,    -inf,    -inf],
          [ 0.8750,  0.9062,  0.9375,    -inf,    -inf],
          [ 0.8750,  0.9062,  0.9375,  0.9688,    -inf],
          [ 0.8750,  0.9062,  0.9375,  0.9688,  1.0000]],

         [[ 0.9688,    -inf,    -inf,    -inf,    -inf],
          [ 0.9688,  0.9766,    -inf,    -inf,    -inf],
          [ 0.9688,  0.97